In [ ]:
!pip install --trusted-host=artifactory.oci.oraclecorp.com -i https://artifactory.oci.oraclecorp.com/api/pypi/global-dev-pypi/simple -U oci==2.155.3+preview.1.254

In [ ]:
from vision_service_python_client import ai_service_vision_client
from vision_service_python_client.models import CreateStreamJobDetails, CreateStreamSourceDetails, \
    RtspSourceDetails, ObjectStorageOutputLocation, PublicStreamNetworkAccessDetails
import oci, time

In [ ]:
CONFIG_PROFILE = "DEFAULT"
COMPARTMENT_ID = ""
STREAM_JOB_DISPLAY_NAME = "Python SDK Test"
CAMERA_URL=""
NAMESPACE= ""
BUCKET= ""
PREFIX= ""
FEATURES = [
        {
            "featureType": "FACE_DETECTION",
            "minConfidence": 0.4
        }
    ]
print("done")

In [ ]:
config = oci.config.from_file('~/.oci/config', profile_name=CONFIG_PROFILE)
endpoint = ""
token_file = config['security_token_file']
with open(token_file, 'r') as f:
   token = f.read()
private_key = oci.signer.load_private_key_from_file(config['key_file'])
signer = oci.auth.signers.SecurityTokenSigner(token, private_key)
ai_service_vision_client = ai_service_vision_client.AIServiceVisionClient(config=config, signer=signer, service_endpoint=endpoint)
print("done")

In [ ]:
create_stream_source_details = CreateStreamSourceDetails()

stream_source_details = RtspSourceDetails()
stream_source_details.camera_url = CAMERA_URL
stream_source_details.stream_network_access_details = PublicStreamNetworkAccessDetails()
create_stream_source_details.stream_source_details = stream_source_details
create_stream_source_details.compartment_id = COMPARTMENT_ID

# Send request for adding the device and parse the response
print("done")

In [ ]:
stream_source = ai_service_vision_client.create_stream_source(create_stream_source_details)
source_id = stream_source.data.id
print(source_id)
while stream_source.data.lifecycle_state != 'ACTIVE':
    stream_source = ai_service_vision_client.get_stream_source(source_id)
    time.sleep(2)
print("done")

In [ ]:
stream_source = ai_service_vision_client.get_stream_source(source_id)
print(stream_source.data)

In [ ]:
create_stream_job_details = CreateStreamJobDetails()
create_stream_job_details.stream_source_id = source_id

# For Object Tracker (Only "face" supported as of now) - commend if not required
create_stream_job_details.features = FEATURES
# print(create_stream_job_detai)

output_location = ObjectStorageOutputLocation()
output_location.namespace_name = NAMESPACE
output_location.bucket_name = BUCKET
output_location.prefix = PREFIX
# Choose output location in object storage, make sure you have created the bucket
create_stream_job_details.stream_output_location = output_location

create_stream_job_details.compartment_id = COMPARTMENT_ID
create_stream_job_details.display_name = STREAM_JOB_DISPLAY_NAME


In [ ]:
stream_job = ai_service_vision_client.create_stream_job(create_stream_job_details=create_stream_job_details)
job_id = stream_job.data.id
print(job_id)
while stream_job.data.lifecycle_state == 'CREATING':
    stream_job = ai_service_vision_client.get_stream_job(job_id)
    time.sleep(10)


In [ ]:
print(ai_service_vision_client.get_stream_job(job_id).data)

In [ ]:
print("Calling startStream job at ", time.time())
ai_service_vision_client.start_stream_job(job_id)
stream_job = ai_service_vision_client.get_stream_job(job_id)

while stream_job.data.lifecycle_state == 'UPDATING':
    print("updating")
    print(stream_job.data.lifecycle_state)
    stream_job = ai_service_vision_client.get_stream_job(job_id)
    time.sleep(2)
print("Complete")


In [ ]:
ai_service_vision_client.stop_stream_job(job_id)
stream_job = ai_service_vision_client.get_stream_job(job_id)

while stream_job.data.lifecycle_state == 'UPDATING':
    stream_job = ai_service_vision_client.get_stream_job(job_id)
    time.sleep(2)

In [ ]:
print(job_id)
ai_service_vision_client.delete_stream_job(job_id)
stream_job = ai_service_vision_client.get_stream_job(job_id)
while stream_job.data.lifecycle_state == 'DELETING':
    stream_job = ai_service_vision_client.get_stream_job(job_id)
    time.sleep(2)

In [ ]:
print(stream_source.data.id)
ai_service_vision_client.delete_stream_source(stream_source.data.id)
stream_job = ai_service_vision_client.get_stream_job(job_id)

while stream_job.data.lifecycle_state == 'DELETING':
    stream_job = ai_service_vision_client.get_stream_source(stream_source.data.id)
    time.sleep(2)